In [ ]:
import pandas as pd
import string
import re
import numpy as np
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import cohen_kappa_score, make_scorer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop_words = set(stopwords.words('english'))

train = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
test = pd.read_csv('../input/quora-insincere-questions-classification/test.csv')
sample_submission = pd.read_csv('../input/quora-insincere-questions-classification/sample_submission.csv')

In [ ]:
#base desbalanceada, precisamos corrigir

train_balanced = train[train['target'] == 1]
train_balanced = train_balanced.append(train[train['target'] == 0].sample(n = len(train_balanced))).reset_index(drop = True)
train_balanced['target'].value_counts()

In [ ]:
def clean(sen):
    text = sen.lower()
  
    tokenized_word = word_tokenize(text)
    ps = PorterStemmer()
    stemmed_words=[]
    for w in tokenized_word:
        stemmed_words.append(ps.stem(w))

    text = ' '.join(stemmed_words)

    return text

train_text = train_balanced['question_text'].apply(lambda text: clean(text))
train_target = train_balanced['target'].values

test_text = test['question_text'].apply(lambda text: clean(text))

In [ ]:
vectorizer = TfidfVectorizer(use_idf=True, ngram_range=(1,2))
tfidf_model = vectorizer.fit(train_text)
train_tfidf = tfidf_model.transform(train_text)
test_tfidf = tfidf_model.transform(test_text)

In [ ]:
model = MultinomialNB()
clf = model.fit(train_tfidf, train_target)

In [ ]:
targets = clf.predict(test_tfidf)

In [ ]:
targets = pd.Series(targets)
subm_df = pd.read_csv('../input/quora-insincere-questions-classification/sample_submission.csv')
subm_df['prediction'] = targets
subm_df.to_csv('submission.csv', index=False)